In [2]:
import pandas as pd
from itertools import product
from tqdm.notebook import tqdm

import sys, os

sys.path.append(os.path.abspath('..'))
%load_ext autoreload
%autoreload 2
from modules.config import *
from modules.storage import (
    get_demand_model_data,
    store_results
)
from modules.svm import *

In [2]:
def get_svm_metas():
    return [
        {'kernel': ['linear'], 'C': [0.1, 1, 10, 100],  'gamma': [-1],                          'degree': [-1],         'max_iter': [1000000]},
        {'kernel': ['rbf'],    'C': [0.1, 1, 10, 100],  'gamma': [0.1, 0.01, 0.001, 0.0001],    'degree': [-1],         'max_iter': [1000000]},
        {'kernel': ['poly'],   'C': [1, 10, 100, 1000], 'gamma': [-1],                          'degree': [2, 3, 4, 5], 'max_iter': [1000000]}
    ]

In [3]:
def execute_stage(path, h3_res, time_interval_length, get_available_model_metas_for_stage, do_evaluate_model, silent):
    all_possible_metas = get_svm_metas()
    metas = get_available_model_metas_for_stage(h3_res, time_interval_length, all_possible_metas)
    
    iterator = metas if silent else tqdm(metas)
    for param_grid in iterator:
        if not silent:
            feedback = f"h3: {h3_res} | t:{time_interval_length} | - " + param_grid[0]["kernel"][0]
            tqdm.write(feedback, end="\r")
        
        model_data_train, model_data_test = get_demand_model_data(h3_res, time_interval_length)
        if len(model_data_train) > 40000:
            model_data_train = model_data_train.sample(40000)

        X_train, X_test, y_train, y_test = split_and_scale_data(model_data_train, model_data_test)
        models = train_model(param_grid, X_train, y_train)
        results = get_results(models, h3_res, time_interval_length, do_evaluate_model, X_test, y_test)
        store_results(results, path)  
        
        if not silent:
            tqdm.write(feedback + " ✓")

In [4]:
execute_stage(
    SVM_FIRST_STAGE_RESULTS_PATH,
    TUNE_H3_RESOLUTION,
    TUNE_TIME_INTERVAL_LENGTH,
    get_availabe_models_metas_first_stage,
    do_evaluate_model=False,
    silent=False,
)

  0%|          | 0/1 [00:00<?, ?it/s]

h3: 8 | t:6 | - linear ✓


In [12]:
results = pd.read_parquet(SVM_FIRST_STAGE_RESULTS_PATH)
results.sort_values(by=['mean_test_score'], ascending=False).head(5)

,iter,n_resources,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,param_max_iter,params,...,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score,n_iter,h3_res,time_interval_length,param_gamma,param_degree
0,0,8290,54.808515,0.629360,11.699974,0.465198,1.0,rbf,1000000,"{'C': 1.0, 'degree': None, 'gamma': 0.01, 'ker...",...,-1.240730,-1.419944,-1.493995,-1.319479,0.116704,434001,8,6,0.010,NaN
3,0,8290,61.887783,2.143711,13.117010,0.575450,10.0,rbf,1000000,"{'C': 10.0, 'degree': None, 'gamma': 0.01, 'ke...",...,-0.968602,-1.101078,-1.162574,-1.023994,0.091734,0,8,6,0.010,NaN
7,0,8290,61.977852,1.651096,8.008226,0.643548,100.0,rbf,1000000,"{'C': 100.0, 'degree': None, 'gamma': 0.001, '...",...,-1.251774,-1.431241,-1.494112,-1.327835,0.114344,0,8,6,0.001,NaN
17,0,8290,72.496359,2.332309,11.504219,0.713604,10.0,poly,1000000,"{'C': 10.0, 'degree': 2.0, 'gamma': None, 'ker...",...,-1.278043,-1.453530,-1.513180,-1.352580,0.110961,0,8,6,NaN,2.0
12,2,74610,9881.552610,513.575073,284.903122,5.123617,10.0,rbf,1000000,"{'C': 10.0, 'degree': None, 'gamma': 0.01, 'ke...",...,-1.367001,-1.403957,-1.402731,-1.397316,0.020037,0,8,6,0.010,NaN


In [13]:
results.sort_values(by='mean_test_score', ascending=False).groupby('param_kernel').mean_test_score.first()

param_kernel
linear   -1.662285
poly     -1.508812
rbf      -1.465577
Name: mean_test_score, dtype: float64

In [14]:
results.groupby('param_kernel').mean_fit_time.sum() / 60

param_kernel
linear    214.667233
poly      476.428271
rbf       949.590948
Name: mean_fit_time, dtype: float64

In [4]:
resolutions = list(product(PREDICTIVE_H3_RESOLUTIONS, CALC_TIME_INTERVAL_LENGTHS))
resolutions = resolutions + (ADDITIONAL_PREDICTIVE_RESOLUTIONS)
resolutions

[(7, 1), (7, 2), (7, 6), (7, 24), (8, 1), (8, 2), (8, 6), (8, 24), (9, 24)]

In [5]:
for h3_res, time_interval_length in tqdm(resolutions):
    execute_stage(
        SVM_SECOND_STAGE_RESULTS_PATH,
        h3_res,
        time_interval_length,
        get_availabe_models_metas_second_stage,
        do_evaluate_model=True,
        silent=True,
    )

  0%|          | 0/9 [00:00<?, ?it/s]

c:\Dev\miniconda\envs\AAA_MAGMA\lib\site-packages\sklearn\svm\_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=1000000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Dev\miniconda\envs\AAA_MAGMA\lib\site-packages\sklearn\svm\_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=1000000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


In [3]:
results = pd.read_parquet(SVM_SECOND_STAGE_RESULTS_PATH)
results.sort_values(by=['mean_test_score'], ascending=False)

,iter,n_resources,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,param_max_iter,...,split4_train_score,mean_train_score,std_train_score,n_iter,h3_res,time_interval_length,mse,mae,mape,rmse
4,0,50000,9384.699127,326.628205,122.565240,18.126016,100.0,0.01,rbf,1000000,...,-0.113932,-0.115442,0.002974,1000000,8,1,0.401745,0.393298,0.331632,0.633833
0,0,50000,2384.510185,30.335371,244.337624,220.820427,100.0,0.01,rbf,1000000,...,-3.917312,-4.009989,0.076132,1000000,7,1,5.392942,1.330544,0.584372,2.322271
1,0,50000,629.624117,7.623112,66.520342,0.718597,100.0,0.01,rbf,1000000,...,-12.218913,-12.179547,0.137815,987825,7,2,14.204874,1.993245,0.638633,3.768935
2,0,50000,437.209460,4.289007,66.760025,0.330587,100.0,0.01,rbf,1000000,...,-68.567081,-69.473354,0.848311,685182,7,6,75.795420,4.009716,0.733445,8.706057
3,0,23710,74.574306,1.281584,14.601953,0.169079,100.0,0.01,rbf,1000000,...,-526.806997,-525.792169,8.815393,142777,7,24,570.006794,10.383714,0.856827,23.874815


In [4]:
results.columns

Index(['iter', 'n_resources', 'mean_fit_time', 'std_fit_time',
       'mean_score_time', 'std_score_time', 'param_C', 'param_gamma',
       'param_kernel', 'param_max_iter', 'params', 'split0_test_score',
       'split1_test_score', 'split2_test_score', 'split3_test_score',
       'split4_test_score', 'mean_test_score', 'std_test_score',
       'rank_test_score', 'split0_train_score', 'split1_train_score',
       'split2_train_score', 'split3_train_score', 'split4_train_score',
       'mean_train_score', 'std_train_score', 'n_iter', 'h3_res',
       'time_interval_length', 'mse', 'mae', 'mape', 'rmse'],
      dtype='object')